In [6]:
import requests
import pandas as pd
import pymysql
import numpy as np
from datetime import datetime
import traceback2

from selenium import webdriver
import time
from bs4 import BeautifulSoup as soup
from selenium.webdriver.common.keys import Keys
import  re



stable_coins = "USDT, USDC, DAI, BUSD, SUSD, UST, TUSD".split(",")

frams_url = "https://pancakeswap.finance/farms"

In [31]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_pancakswap_data_df(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["Date"].timestamp()
        try:
            pair2 = row["Pair_2"] if row["Pair_2"] else ' '
            print(row["Date"], row["Pair_1"], pair2, row["Pool_rate"], row["pool_tvl"], row["Farm_payment"], row["Where"], row["includes_stablecoin"])
            sql = "INSERT INTO `pancakswap_farm_pool_data` (`timestamp_dt`, `pair1`,`pair2`,`pool_rate`,`pool_size`,`farm_payment`,`where`,`includes_stablecoin`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, row["Pair_1"], pair2, row["Pool_rate"], row["pool_tvl"], row["Farm_payment"], row["Where"], row["includes_stablecoin"]))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()

In [12]:
def get_selenium_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument("no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument('--headless')
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--enable-javascript")
    browser = webdriver.Chrome("chromedriver", options=options)
    return browser


def extract_data(page_source):
    df_c =  pd.DataFrame(columns = ['name','earned', 'apr', 'liquidity', 'multiplier'])
    
    page_soup = soup(page_source, "html.parser")
    table = page_soup.findAll("table")#, {"class": "sc-hRMXGO hbbpxV"})
    #heads = table[0].thead.tr.findAll("th")
    rows =  table[0].tbody.findAll("tr")
    for row in rows: 
#         print(row)
        divs = row.findAll("div", {"color": "text"})
        name = divs[0].text
        #name
        earned = row.find_all("span")[0].text
        #earned
        apr = row.find_all(text=re.compile('%'))[0]
        #apr
        liq = divs[1].text
        #liq
        mul = row.find_all(text=re.compile('x'))[0]
        #mul
        #print(f'{name} {earned} {apr} {liq} {mul}')
        df_c.at[len(df_c)] = [name,earned,apr,liq,mul]
    return df_c


def crawl_page(browser, url):
    browser.get(url)
#     time.sleep(5)
    prev_len = 0
    curr_len = 100
    count = 0
    page_source = None
    while count<5:
        print(curr_len)
#         time.sleep(3)
        page_source = browser.page_source
        #df = extract_data(page_source)
#         time.sleep(3)
        #print(df)
        body = browser.find_element_by_css_selector('body')
        body.send_keys(Keys.PAGE_DOWN)
#         time.sleep(1)
        body.send_keys(Keys.PAGE_DOWN)
        curr_len = len(page_source)
        if curr_len == prev_len:
            count = count+1
        else:
            count = 0
        prev_len = curr_len
    return browser.page_source

def get_page_as_df():
    browser = get_selenium_browser()
    page_source = crawl_page(browser, frams_url)
    df = extract_data(page_source)
    
    return df

In [13]:
df = get_page_as_df()

100
126362
126401
126401
180893
235476
235628
290336
344668
399269
399534
453896
508469
508584
563122
617733
617882
617882
617882
617882
617882


In [21]:
def add_columns():
    df[['Pair_1', 'Pair_2']] = df['name'].str.split("-", expand=True)[[0,1]]
    df['Farm_payment'] = 'CAKE'
    df['Where'] = 'PancakeSwap'
    df['includes_stablecoin'] = (df['Pair_1'].isin(stable_coins)) | (df['Pair_2'].isin(stable_coins))
    df['pool_tvl'] = df['liquidity'].apply(lambda x: x.strip('$').replace(',','',-1))
    df['Pool_rate'] = df['apr'].apply(lambda x: x.strip('%').replace(',','',-1))
    now = datetime.now()
    now = datetime(now.year, now.month, now.day)
    df['Date'] = now


In [39]:
df

,name,earned,apr,liquidity,multiplier,Pair_1,Pair_2,Farm_payment,Where,includes_stablecoin,Date,pool_size,pool_rate,pool_tvl,Pool_rate
0,CAKE-BNB,0,60.5%,"$1,590,506,850",40x,CAKE,BNB,CAKE,PancakeSwap,False,2021-05-10,"1,590,506,850",60.5,1590506850,60.5
1,BUSD-BNB,0,20.92%,"$919,835,619",8x,BUSD,BNB,CAKE,PancakeSwap,False,2021-05-10,"919,835,619",20.92,919835619,20.92
2,MCOIN-UST,0,520.42%,"$4,622,350",1x,MCOIN,UST,CAKE,PancakeSwap,False,2021-05-10,"4,622,350",520.42,4622350,520.42
3,WIN-BNB,0,147.7%,"$16,286,877",1x,WIN,BNB,CAKE,PancakeSwap,False,2021-05-10,"16,286,877",147.7,16286877,147.7
4,TRX-BNB,0,60.57%,"$39,716,218",1x,TRX,BNB,CAKE,PancakeSwap,False,2021-05-10,"39,716,218",60.57,39716218,60.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,LINK-BNB,0,33.52%,"$71,765,617",1x,LINK,BNB,CAKE,PancakeSwap,False,2021-05-10,"71,765,617",33.52,71765617,33.52
116,EOS-BNB,0,19.02%,"$12,644,868",0.1x,EOS,BNB,CAKE,PancakeSwap,False,2021-05-10,"12,644,868",19.02,12644868,19.02
117,DOT-BNB,0,25.54%,"$94,192,066",1x,DOT,BNB,CAKE,PancakeSwap,False,2021-05-10,"94,192,066",25.54,94192066,25.54
118,BAND-BNB,0,64.55%,"$37,266,349",1x,BAND,BNB,CAKE,PancakeSwap,False,2021-05-10,"37,266,349",64.55,37266349,64.55


In [40]:
insert_pancakswap_data_df(df)

2021-05-10 00:00:00 CAKE BNB 60.5 1590506850 CAKE PancakeSwap False
2021-05-10 00:00:00 BUSD BNB 20.92 919835619 CAKE PancakeSwap False
2021-05-10 00:00:00 MCOIN UST 520.42 4622350 CAKE PancakeSwap False
2021-05-10 00:00:00 WIN BNB 147.7 16286877 CAKE PancakeSwap False
2021-05-10 00:00:00 TRX BNB 60.57 39716218 CAKE PancakeSwap False
2021-05-10 00:00:00 BTT BNB 92 26146381 CAKE PancakeSwap False
2021-05-10 00:00:00 PNT PBTC 303.96 7914165 CAKE PancakeSwap False
2021-05-10 00:00:00 XMARK BUSD 143.49 16764539 CAKE PancakeSwap False
2021-05-10 00:00:00 BTCB BUSD 17.81 135067787 CAKE PancakeSwap False
2021-05-10 00:00:00 LMT BNB 242.1 9936413 CAKE PancakeSwap False
2021-05-10 00:00:00 DFD BUSD 238.67 10079103 CAKE PancakeSwap False
2021-05-10 00:00:00 ALPACA BUSD 40.28 59723428 CAKE PancakeSwap False
2021-05-10 00:00:00 ΤBTC BTCB 48.87 49224953 CAKE PancakeSwap False
'charmap' codec can't encode character '\u03a4' in position 183: character maps to <undefined>
2021-05-10 00:00:00 SWINGBY B

In [43]:
import pandas as pd
import pymysql
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup as soup
from selenium.webdriver.common.keys import Keys
import re

stable_coins = "USDT, USDC, DAI, BUSD, SUSD, UST, TUSD".split(",")

frams_url = "https://pancakeswap.finance/farms"


def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()


def insert_pancakswap_data_df(df):
    df = df.astype(object).where(pd.notnull(df), None)
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        dt = row["Date"].timestamp()
        try:
            pair2 = row["Pair_2"] if row["Pair_2"] else ' '
            print(row["Date"], row["Pair_1"], pair2, row["Pool_rate"], row["pool_tvl"], row["Farm_payment"], row["Where"], row["includes_stablecoin"])
            sql = "INSERT INTO `pancakswap_farm_pool_data` (`timestamp_dt`, `pair1`,`pair2`,`pool_rate`,`pool_size`,`farm_payment`,`where`,`includes_stablecoin`) VALUES (FROM_UNIXTIME(%s), %s, %s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (dt, row["Pair_1"], pair2, row["Pool_rate"], row["pool_tvl"], row["Farm_payment"], row["Where"], row["includes_stablecoin"]))
        except Exception as e:
            print(e)
    connection.commit()
    cursor.close()
    connection.close()


def get_selenium_browser():
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument("no-sandbox")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1200,900")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument('--headless')
    options.add_argument("start-maximized")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    options.add_argument("--disable-blink-features")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--enable-javascript")
    browser = webdriver.Chrome("chromedriver", options=options)
    return browser


def extract_data(page_source):
    df_c = pd.DataFrame(columns=['name', 'earned', 'apr', 'liquidity', 'multiplier'])
    page_soup = soup(page_source, "html.parser")
    table = page_soup.findAll("table")
    rows = table[0].tbody.findAll("tr")
    for row in rows:
        divs = row.findAll("div", {"color": "text"})
        name = divs[0].text
        earned = row.find_all("span")[0].text
        apr = row.find_all(text=re.compile('%'))[0]
        liq = divs[1].text
        mul = row.find_all(text=re.compile('x'))[0]
        df_c.at[len(df_c)] = [name, earned, apr, liq, mul]
    return df_c


def crawl_page(browser, url):
    browser.get(url)
#     time.sleep(5)
    prev_len = 0
    curr_len = 100
    count = 0
    page_source = None
    while count < 5:
        print(curr_len)
#         time.sleep(3)
        page_source = browser.page_source
        body = browser.find_element_by_css_selector('body')
        body.send_keys(Keys.PAGE_DOWN)
#         time.sleep(1)
        body.send_keys(Keys.PAGE_DOWN)
        curr_len = len(page_source)
        if curr_len == prev_len:
            count = count + 1
        else:
            count = 0
        prev_len = curr_len
    return browser.page_source


def get_page_as_df():
    browser = get_selenium_browser()
    page_source = crawl_page(browser, frams_url)
    df = extract_data(page_source)
    return df


def add_columns(df):
    df[['Pair_1', 'Pair_2']] = df['name'].str.split("-", expand=True)[[0, 1]]
    df['Farm_payment'] = 'CAKE'
    df['Where'] = 'PancakeSwap'
    df['includes_stablecoin'] = (df['Pair_1'].isin(stable_coins)) | (df['Pair_2'].isin(stable_coins))
    df['pool_tvl'] = df['liquidity'].apply(lambda x: x.strip('$').replace(',', '', -1))
    df['Pool_rate'] = df['apr'].apply(lambda x: x.strip('%').replace(',', '', -1))
    now = datetime.now()
    now = datetime(now.year, now.month, now.day)
    df['Date'] = now
    return df


def update_data():
    df = get_page_as_df()
    df = add_columns(df)
    insert_pancakswap_data_df(df)


if __name__ == "__main__":
    update_data()


100
126367
126406
126406
180897
235482
235634
290110
344677
344715
399352
453906
508748
563322
617894
617894
617894
617894
617894
2021-05-10 00:00:00 CAKE BNB 60.63 1594404339 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-CAKE-BNB' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 BUSD BNB 21.05 918449569 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-BUSD-BNB' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 MCOIN UST 513.02 4711073 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-MCOIN-UST' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 WIN BNB 149.26 16192381 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-WIN-BNB' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 TRX BNB 61.9 39046567 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-TRX-BNB' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 BTT BNB 9

(1062, "Duplicate entry '2021-05-10 00:00:00-FRONT-BNB' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 HELMET BNB 36.62 33003939 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-HELMET-BNB' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 BTCST BNB 105.79 22845996 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-BTCST-BNB' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 LTC BNB 13.12 18423312 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-LTC-BNB' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 USDC BUSD 11.85 102000904 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-USDC-BUSD' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 DAI BUSD 13.52 89370193 CAKE PancakeSwap False
(1062, "Duplicate entry '2021-05-10 00:00:00-DAI-BUSD' for key 'pancakswap_farm_pool_data.PRIMARY'")
2021-05-10 00:00:00 BSCX BNB 171.49 14093088 CAKE 